In [3]:
import pandas as pd
import numpy as np

from statsmodels.discrete.discrete_model import Logit
from sklearn.linear_model import LogisticRegression
from scipy.special import logit

import seaborn as sns
import matplotlib.pyplot as plt
from netCDF4 import Dataset


In [72]:
# Library to work with netCDF files
# https://github.com/Unidata/netcdf4-python/blob/master/examples/reading_netCDF.ipynb

file_path = "../data/gebco_2023_n51.5918_s45.022_w-68.4229_e-57.6123.nc"
# Open a .nc file ("file_name")
dataset = Dataset(file_path)

In [15]:
for d in dataset.dimensions.items():
    print(d)
dataset.variables

('lat', <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 1577)
('lon', <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 2594)


{'lat': <class 'netCDF4._netCDF4.Variable'>
 float64 lat(lat)
     standard_name: latitude
     long_name: latitude
     units: degrees_north
     axis: Y
     sdn_parameter_urn: SDN:P01::ALATZZ01
     sdn_parameter_name: Latitude north
     sdn_uom_urn: SDN:P06::DEGN
     sdn_uom_name: Degrees north
 unlimited dimensions: 
 current shape = (1577,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'lon': <class 'netCDF4._netCDF4.Variable'>
 float64 lon(lon)
     standard_name: longitude
     long_name: longitude
     units: degrees_east
     axis: X
     sdn_parameter_urn: SDN:P01::ALONZZ01
     sdn_parameter_name: Longitude east
     sdn_uom_urn: SDN:P06::DEGE
     sdn_uom_name: Degrees east
 unlimited dimensions: 
 current shape = (2594,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'elevation': <class 'netCDF4._netCDF4.Variable'>
 int16 elevation(lat, lon)
     standard_name: height_above_mean_sea_level
     long_name: Elevation relative to sea level

In [22]:
elevation = dataset.variables["elevation"]
elevation.dimensions


('lat', 'lon')

In [21]:
elevation.shape


(1577, 2594)

In [23]:
elevation.units

'm'

In [67]:
lon_array, lat_array = dataset.variables['lat'], dataset.variables['lon']
print(lon_array)
print(lat_array)

<class 'netCDF4._netCDF4.Variable'>
float64 lat(lat)
    standard_name: latitude
    long_name: latitude
    units: degrees_north
    axis: Y
    sdn_parameter_urn: SDN:P01::ALATZZ01
    sdn_parameter_name: Latitude north
    sdn_uom_urn: SDN:P06::DEGN
    sdn_uom_name: Degrees north
unlimited dimensions: 
current shape = (1577,)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
float64 lon(lon)
    standard_name: longitude
    long_name: longitude
    units: degrees_east
    axis: X
    sdn_parameter_urn: SDN:P01::ALONZZ01
    sdn_parameter_name: Longitude east
    sdn_uom_urn: SDN:P06::DEGE
    sdn_uom_name: Degrees east
unlimited dimensions: 
current shape = (2594,)
filling on, default _FillValue of 9.969209968386869e+36 used


In [26]:
# extract lat/lon values (in degrees) to numpy arrays
latvals = lat[:]; lonvals = lon[:] 
# a function to find the index of the point closest pt
# (in squared distance) to give lat/lon value.
def getclosest_ij(lats,lons,latpt,lonpt):
    # find squared distance of every point on grid
    dist_sq = (lats-latpt)**2 + (lons-lonpt)**2  
    # 1D index of minimum dist_sq element
    minindex_flattened = dist_sq.argmin()    
    # Get 2D index for latvals and lonvals arrays from 1D index
    return np.unravel_index(minindex_flattened, lats.shape)

iy_min, ix_min = getclosest_ij(latvals, lonvals, 48, -64)

ValueError: operands could not be broadcast together with shapes (1577,) (2594,) 

In [71]:
i = np.abs(lon_array[:] - 46.53).argmin()
j = np.abs(lat_array[:] - -64.67).argmin()
i, j
elevation[i,j]

masked_array(data=-5,
             mask=False,
       fill_value=999999,
            dtype=int16)

In [74]:
import xarray as xr

dataset = xr.open_dataset(file_path)

dataset


<xarray.Dataset> Size: 8MB
Dimensions:    (lat: 1577, lon: 2594)
Coordinates:
  * lat        (lat) float64 13kB 45.02 45.03 45.03 45.04 ... 51.58 51.59 51.59
  * lon        (lon) float64 21kB -68.42 -68.41 -68.41 ... -57.62 -57.62 -57.61
Data variables:
    elevation  (lat, lon) int16 8MB ...
Attributes: (12/36)
    title:                           The GEBCO_2023 Grid - a continuous terra...
    summary:                         The GEBCO_2023 Grid is a continuous, glo...
    keywords:                        BATHYMETRY/SEAFLOOR TOPOGRAPHY, DIGITAL ...
    Conventions:                     CF-1.6, ACDD-1.3
    id:                              DOI: 10.5285/f98b053b-0cbc-6c23-e053-6c8...
    naming_authority:                https://dx.doi.org
    ...                              ...
    geospatial_vertical_units:       meters
    geospatial_vertical_resolution:  1.0
    geospatial_vertical_positive:    up
    identifier_product_doi:          DOI: 10.5285/f98b053b-0cbc-6c23-e053-6c8...
    references:                      DOI: 10.5285/f98b053b-0cbc-6c23-e053-6c8...
    node_offset:                     1.0

In [96]:
lat=47.87
lon=-60.24

subset = dataset.sel(lat=lat,lon=lon, method="nearest")
print(f"""
the elevation at ({lat},{lon}) is {subset.elevation.values}{subset.elevation.units}. 
""")
    
    


the elevation at (47.87,-60.24) is -492m. 
